<a href="https://colab.research.google.com/github/mugalan/energy-plus-utility/blob/dev/BMS_Cookbook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install

In [ ]:
# install from your dev branch
!pip install -q "energy-plus-utility @ git+https://github.com/mugalan/energy-plus-utility.git@dev"

In [ ]:
# run the silent bootstrap in this kernel
from eplus import prepare_colab_eplus
prepare_colab_eplus()  # raises on failure, otherwise silent

# now your imports work
from eplus import EPlusUtil, EPlusSqlExplorer

## Initialize class

In [ ]:
import subprocess, json, pathlib, os
import pandas as pd
EPLUS = str(pathlib.Path.home() / "EnergyPlus-25-1-0")
EPLUS_ROOT = "/root/EnergyPlus-25-1-0"

out_dir = "/content/eplus_out"
idf = f"{EPLUS}/ExampleFiles/5ZoneAirCooled.idf"
epw = f"{EPLUS}/WeatherData/USA_CA_San.Francisco.Intl.AP.724940_TMY3.epw"

util = EPlusUtil(verbose=1, out_dir = out_dir)
util.delete_out_dir()
util.set_model(idf,epw)
state = util.api.state_manager.new_state()
util.ensure_output_sqlite()

In [ ]:
def test_method (self,s,aa):
    print('inside test',aa)
    return aa

In [ ]:
import types
util.test_method = types.MethodType(test_method, util)

In [ ]:
util.test_method(0,1)

## *Optional:* Convert the model to .json

In [ ]:
idf_path = pathlib.Path(idf)
converter = os.path.join(EPLUS_ROOT, "ConvertInputFormat")   # on Windows it's ConvertInputFormat.exe

# Convert IDF → epJSON (outputs 5ZoneAirCooled.epJSON in the same folder)
subprocess.run([converter, str(idf_path)], check=True)

epjson_path = idf_path.with_suffix(".epJSON")
print("epJSON exists?", epjson_path.exists(), epjson_path)

## Simulations

###Dry run to create tables etc.

In [ ]:
util = EPlusUtil(verbose=1)
util.delete_out_dir()
util.set_model(idf,epw)
util.ensure_output_sqlite()
util.dry_run_min(include_ems_edd=False)

In [ ]:
util.list_zone_names()

In [ ]:
pd.DataFrame(util.list_variables_safely())

In [ ]:
pd.DataFrame(util.list_actuators_safely())

## Setup Simulation

### Enable Varaibles and Meters

In [ ]:
# 1) Ensure the meter(s) you want are reported
output_meteres = ["InteriorLights:Electricity:Zone:SPACE5-1","Cooling:EnergyTransfer:Zone:SPACE1-1","Cooling:EnergyTransfer","Electricity:Facility","ElectricityPurchased:Facility", "ElectricitySurplusSold:Facility"]

util.ensure_output_meters(output_meteres, freq="TimeStep")


### Run Simulation

In [ ]:
# util = EPlusUtil(verbose=1)
util.delete_out_dir()
util.set_model(idf,epw, outdoor_co2_ppm=400.0, per_person_m3ps_per_W=3.82e-08)
# util.ensure_output_variables([
#     # {"name":"Site Outdoor Air Drybulb Temperature", "key":"", "freq":"TimeStep"},
#     # {"name":"Site Outdoor Air Dewpoint Temperature", "key":"", "freq":"TimeStep"},
#     # {"name":"Site Wind Speed", "key":"", "freq":"TimeStep"},
#     # {"name":"Site Diffuse Solar Radiation Rate per Area", "key":"", "freq":"TimeStep"},
#     # {"name":"Site Direct Solar Radiation Rate per Area", "key":"", "freq":"TimeStep"},
#     {'name':'Zone Mean Air Temperature',          'key':'*',           'freq':'TimeStep'},
#     {'name':'Zone Mean Air Dewpoint Temperature', 'key':'*',           'freq':'TimeStep'},
#     {'name':'Zone Air Relative Humidity',          'key':'*',           'freq':'TimeStep'},
#     {'name':'Zone People Occupant Count',         'key':'*',           'freq':'TimeStep'},
#     {'name':'Site Outdoor Air Drybulb Temperature','key':'Environment','freq':'TimeStep'},
#     {'name':'Site Outdoor Air Wetbulb Temperature','key':'Environment','freq':'TimeStep'},
#     {"name":"Zone Air System Sensible Cooling Energy", "key":"*", "freq":"TimeStep"},
#     {"name":"Zone Total Internal Latent Gain Energy", "key":"*", "freq":"TimeStep"},
#     {"name": "Zone Air CO2 Concentration", "key": "*", "freq": "TimeStep"},
#     {"name": "Zone Outdoor Air Inlet Mass Flow Rate", "key": "*", "freq": "TimeStep"},
#     {"name": "System Node Standard Density Volume Flow Rate", "key": "*", "freq": "TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE2-1", "freq":"TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE3-1", "freq":"TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE4-1", "freq":"TimeStep"},
#     # {"name":"Zone Air System Sensible Cooling Energy", "key":"SPACE5-1", "freq":"TimeStep"},
# ], activate=True)
# 1) Make sure SQL will be produced
# util.ensure_output_sqlite(activate=True)

# 2) Variables to record (TimeStep)
specs = [
    # --- Zone state + people ---
    {"name": "Zone Mean Air Temperature",                "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Mean Air Dewpoint Temperature",       "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Air Relative Humidity",               "key": "*",            "freq": "TimeStep"},
    {"name": "Zone Mean Air Humidity Ratio",             "key": "*",            "freq": "TimeStep"},
    {"name": "Zone People Occupant Count",               "key": "*",            "freq": "TimeStep"},

    # --- CO₂ & OA into zones ---
    {"name": "Zone Air CO2 Concentration",               "key": "*",            "freq": "TimeStep"},

    # --- Site weather (Environment key) ---
    {"name": "Site Outdoor Air Drybulb Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Wetbulb Temperature",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Relative Humidity",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Humidity Ratio",     "key": "Environment",  "freq": "TimeStep"},
    {"name": "Site Outdoor Air Barometric Pressure",        "key": "Environment", "freq": "TimeStep"},
    {"name": "Site Outdoor Air CO2 Concentration",                          "key": "Environment",  "freq": "TimeStep"},

    {"name": "System Node Temperature",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node Mass Flow Rate",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node Humidity Ratio",               "key": "*",            "freq": "TimeStep"},
    {"name": "System Node CO2 Concentration",               "key": "*",            "freq": "TimeStep"},
]


    # # --- Air system OA (scalar, per air loop) ---
    # {"name": "Air System Outdoor Air Mass Flow Rate",    "key": "",             "freq": "TimeStep"},
    # {"name": "Air System Outdoor Air Flow Fraction",     "key": "",             "freq": "TimeStep"},

    # # --- Nodes (use * for all; swap to explicit node names if you prefer) ---
    # {"name": "System Node Mass Flow Rate",               "key": "*",            "freq": "TimeStep"},
    # {"name": "System Node Standard Density Volume Flow Rate", "key": "*",       "freq": "TimeStep"},

    # # --- Infiltration (per zone) ---
    # {"name": "Zone Infiltration Standard Density Volume Flow Rate", "key": "*", "freq": "TimeStep"},

# 3) Ensure the Output:Variable objects exist (dedup-aware)
util.ensure_output_variables(specs, activate=True)

util.ensure_output_sqlite()
util.verbose = 1
util.enable_runtime_logging()
# util.register_begin_iteration(["occupancy_counter"], clear=True)
util.register_begin_iteration(
    [{"method_name": "occupancy_counter",
      "kwargs": {"lam": 25.0, "min": 24, "max": 27, "seed": 123}}],
    clear=True,
    run_during_warmup=False,   # set True if you want it during warmup
    enable=True
)
# # register JUST this one method to start
# util.register_begin_iteration([
#     {"method_name": "co2_set_outdoor_ppm_simple",
#      "key_wargs": {"value_ppm": 0.0, "log_every_minutes": 60, "verify": True}}
# ], clear=True, run_during_warmup=False)
# util.register_begin_iteration([
#     {"method_name": "co2_set_outdoor_ppm_debug",
#      "key_wargs": {"value_ppm": 0.0}}
# ], clear=True, run_during_warmup=True)  # run during warmup so you see output throughout
# util.register_begin_iteration([
#     {"method_name": "co2_set_outdoor_ppm",
#      "key_wargs": {"value_ppm": 400.0, "log_every_minutes": None, "verify": True}}
# ], clear=True, run_during_warmup=False)
# util.register_begin_iteration(
#     [{"method_name": "probe_zone_air_and_supply",
#       "kwargs": {"log_every_minutes": 1}}],
#     clear=False,
#     run_during_warmup=False,
#     enable=True
# )
# util.register_after_hvac_reporting(
#     [{"method_name": "probe_zone_air_and_supply",
#       "key_wargs": {"log_every_minutes": 1, "precision": 3}}],
#     clear=False, run_during_warmup=False
# )
# run AFTER HVAC reporting so node/site values are finalized
# util.register_after_hvac_reporting([
#     {"method_name": "api_check_zone_humidity_ratio",
#      "key_wargs": {"max_zones": 5, "precision": 6}}
# ], clear=False, run_during_warmup=False)
util.register_after_hvac_reporting([
    {"method_name": "probe_zone_air_and_supply_with_kf",
     "key_wargs": {
         "log_every_minutes": 15,
         "precision": 3,
         "kf_db_filename": "eplusout_kf_test.sqlite",   # or another file to keep EP DB clean
         "kf_batch_size": 50,
         "kf_commit_every_batches": 10,
         "kf_checkpoint_every_commits": 5,
         "kf_journal_mode": "WAL",
         "kf_synchronous": "NORMAL",
         "kf_nonneg_idx":[3,4]
     }}
], clear=False, run_during_warmup=False)
# util.register_after_hvac_reporting([
#     {"method_name": "kf_dummy_sql_writer_fast",
#      "key_wargs": {
#          "table": "KalmanEstimatesTest",
#          "db_filename": "eplusout_kf_test.sqlite",
#          "batch_size": 100,
#          "commit_every_batches": 5,
#          "checkpoint_every_commits": 2,
#          "log_every_ticks": 1000,          # much less spammy
#          "run_during_warmup": False,
#          "journal_mode": "WAL",
#          "synchronous": "NORMAL"
#      }}
# ], clear=True)
rc=util.run_annual()

In [ ]:
probe_zone_air_and_supply_with_kf

In [ ]:
util.state

### Table Inspect

In [ ]:
xp = EPlusSqlExplorer(f"{out_dir}/eplusout.sql")

In [ ]:
xp.list_tables()

In [ ]:
df = xp.list_sql_variables(name="System Node Temperature")
df[['KeyValue','n_rows']].head(20)

In [ ]:
df.to_dict()

In [ ]:
df

### Analyze weather data

In [ ]:
# util.ensure_output_sqlite()
site_vars = [
    "Site Outdoor Air Drybulb Temperature",
    "Site Outdoor Air Dewpoint Temperature",
    "Site Outdoor Air Humidity Ratio",
    "Site Outdoor Air Barometric Pressure",
    "Site Wind Speed",
    "Site Wind Direction",
    "Site Diffuse Solar Radiation Rate per Area",
    "Site Direct Solar Radiation Rate per Area",
    "Site Horizontal Infrared Radiation Rate per Area",
    "Site Sky Temperature",
]
util.ensure_output_sqlite()
util.ensure_output_variables(
    [{'name': v, 'key': 'Environment', 'freq': 'TimeStep'} for v in site_vars]
)
util.run_annual()

In [ ]:
util.export_weather_sql_to_csv()

In [ ]:
weather_df=pd.read_csv('eplus_out/weather_timeseries.csv')
weather_df['timestamp'] = pd.to_datetime(weather_df['timestamp'])
weather_df['month'] = weather_df['timestamp'].dt.month

In [ ]:
weather_df

In [ ]:
import numpy as np
from scipy.stats import norm, lognorm, gamma
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Extract the temperature data
variable='Site Outdoor Air Humidity Ratio [kgWater/kgDryAir]'#'Site Outdoor Air Dewpoint Temperature [C]' #'Site Outdoor Air Drybulb Temperature [C]' #
n=9
data_df = weather_df[weather_df['month']==n]
data = data_df[variable]

# Fit a normal distribution to the data:
mu, std = norm.fit(data)

# Create the histogram trace from the previous plot
counts, bin_edges = np.histogram(data, bins=50) # Adjust bin count as needed
bin_centers = 0.5 * (bin_edges[:-1] + bin_edges[1:])

histogram_trace = go.Bar(x=bin_centers, y=counts, name='Histogram', opacity=0.7)

# Create the Gaussian curve trace
xmin, xmax = data.min(), data.max()
x_norm = np.linspace(xmin, xmax, 100)
p_norm = norm.pdf(x_norm, mu, std)

# Scale the PDF to match the histogram's count scale
bin_width = bin_edges[1] - bin_edges[0]
scaled_pdf_norm = p_norm * len(data) * bin_width

gaussian_trace = go.Scatter(x=x_norm, y=scaled_pdf_norm, mode='lines', name=f'Gaussian Fit (μ={mu:.2f}, σ={std:.2f})', line=dict(color='red', width=2))

# Fit Log-Normal distribution
# Log-normal distribution requires positive data. Since temperature can be negative,
# a simple log-normal fit might not be appropriate directly.
# However, for demonstration, we can fit it to the positive part or shift the data.
# Let's fit it to the original data, understanding the limitations if negative values exist.
# We need to be careful if temperature_data contains zero or negative values for lognorm fit.
# For simplicity, we'll add an offset if there are non-positive values.
offset = 0
if (data <= 0).any():
    offset = -data.min() + 1 # Shift data to be positive
    print(f"Shifting data by {offset:.2f} for Log-Normal fit to ensure positivity.")

shape_lognorm, loc_lognorm, scale_lognorm = lognorm.fit(data + offset)

# Generate points for the fitted Log-Normal curve
# Ensure the x range is appropriate for the shifted data
x_lognorm = np.linspace(data.min() + offset, data.max() + offset, 100)
p_lognorm = lognorm.pdf(x_lognorm, shape_lognorm, loc_lognorm, scale_lognorm)

# Scale the PDF and shift x back for plotting
scaled_pdf_lognorm = p_lognorm * len(data) * bin_width
x_lognorm_unshifted = x_lognorm - offset

lognormal_trace = go.Scatter(x=x_lognorm_unshifted, y=scaled_pdf_lognorm, mode='lines', name=f'Log-Normal Fit', line=dict(color='green', width=2))


# Fit Gamma distribution
# Gamma distribution also typically requires positive data. Similar consideration as Log-Normal.
# We'll fit it to the shifted data if an offset was applied for lognormal.
shape_gamma, loc_gamma, scale_gamma = gamma.fit(data + offset)

# Generate points for the fitted Gamma curve
# Ensure the x range is appropriate for the shifted data
x_gamma = np.linspace(data.min() + offset, data.max() + offset, 100)
p_gamma = gamma.pdf(x_gamma, shape_gamma, loc_gamma, scale_gamma)

# Scale the PDF and shift x back for plotting
scaled_pdf_gamma = p_gamma * len(data) * bin_width
x_gamma_unshifted = x_gamma - offset


gamma_trace = go.Scatter(x=x_gamma_unshifted, y=scaled_pdf_gamma, mode='lines', name=f'Gamma Fit', line=dict(color='purple', width=2))


# Create the figure and add traces
fig = go.Figure()
fig.add_trace(histogram_trace)
fig.add_trace(gaussian_trace)
fig.add_trace(lognormal_trace)
fig.add_trace(gamma_trace)


# Update layout
fig.update_layout(title=f'Distribution of {variable} with Distribution Fits',
                  xaxis_title=variable,
                  yaxis_title='Count',
                  barmode='overlay' # Overlay bars to see fits better
                 )

# Show the plot
fig.show()

### Plot Results

#### Zone plots

In [ ]:
# Drybulb (auto-picks top zone keys if keys=None)
fig1=util.plot_sql_zone_variable(
    "Zone Mean Air Temperature",
    keys=["*"],                          # auto-pick a few zones with data
    reporting_freq=("TimeStep",),       # match how you logged
    resample="1h",
    title="Zone Mean Air Temperature"
)

# Humidity ratio
fig2=util.plot_sql_zone_variable(
    "Zone Mean Air Humidity Ratio",
    keys=["*"],
    reporting_freq=("TimeStep",),
    resample="1h",
    title="Zone Mean Air Humidity Ratio"
)

# CO2 concentration
fig3=util.plot_sql_zone_variable(
    "Zone Air CO2 Concentration",
    keys={"*"},
    reporting_freq=("TimeStep",),
    resample="1h",
    title="Zone Air CO2 Concentration"
)

#### Outdoor Air Plots

In [ ]:
sels = [
    {"kind":"var", "name":"Site Outdoor Air Drybulb Temperature", "key":"Environment", "label":"Tdb [C]"},
    {"kind":"var", "name":"Site Outdoor Air Humidity Ratio",      "key":"Environment", "label":"w [kg/kg]"},
]
fig4=util.plot_sql_series(
    selections=sels,
    reporting_freq=("TimeStep",),
    include_design_days=False,
    resample="1h",
    meters_to_kwh=False,
    title="Outdoor (Environment)"
)

#### Supply Air Plots

In [ ]:
z2nodes = util._discover_zone_inlet_nodes_from_sql()
zone = "SPACE1-1"
sels = [{"kind":"var", "name":"System Node Mass Flow Rate", "key":n, "label":n} for n in z2nodes[zone]]
fig5=util.plot_sql_series(
    selections=sels,
    reporting_freq=("TimeStep",),
    resample="15min",
    meters_to_kwh=False,
    title=f"{zone} — Supply Node Mass Flow Rate"
)

In [ ]:
zone = "SPACE1-1"
sels = [{"kind":"var", "name":"System Node Temperature", "key":n, "label":n} for n in z2nodes[zone]]
fig6=util.plot_sql_series(selections=sels, reporting_freq=("TimeStep",), resample="15min",
                     meters_to_kwh=False, title=f"{zone} — Supply Node Temperature")

In [ ]:
zone = "SPACE1-1"
sels = [{"kind":"var", "name":"System Node Humidity Ratio", "key":n, "label":n} for n in z2nodes[zone]]
fig7=util.plot_sql_series(selections=sels, reporting_freq=("TimeStep",), resample="15min",
                     meters_to_kwh=False, title=f"{zone} — Supply Node Humidity Ratio")

In [ ]:
zone = "SPACE1-1"
sels = [{"kind":"var", "name":"System Node CO2 Concentration", "key":n, "label":n} for n in z2nodes[zone]]
fig8=util.plot_sql_series(selections=sels, reporting_freq=("TimeStep",), resample="15min",
                     meters_to_kwh=False, title=f"{zone} — Supply Node CO2 Concentration")

#### Covariance plots

In [ ]:
# 1) discover the exact zone keys present for the occupancy variable
occ_keys = (
    util.list_sql_zone_variables(
        name='Zone People Occupant Count',
        reporting_freq=None,              # don't filter; show all
        include_design_days=False
    )['KeyValue']
    .dropna().astype(str).tolist()
)

# 2) build selections using those keys (so keys and zones match the DB)
# output_sels = [
#     {'kind':'var','name':'Zone Mean Air Temperature','key':k,'label':f'MAT: {k}'}
#     for k in occ_keys
# ] + [
#     {'kind':'var','name':'Zone Air Relative Humidity','key':k,'label':f'ARH: {k}'}
#     for k in occ_keys
# ] +
# output_sels =  [
#     {'kind':'var','name':'Zone Air System Sensible Cooling Energy','key':k,'label':f'QSEN: {k}'}
#     for k in occ_keys
# ]
# + [
#     {'kind':'var','name':'Zone Total Internal Latent Gain Energy','key':k,'label':f'QLAT: {k}'}
#     for k in occ_keys
# ]
output_sels = [
    {'kind':'var','name':'Zone Air CO2 Concentration','key':k,'label':f'CO2: {k}'}
    for k in occ_keys
]

control_sels = (
    [{'kind':'var','name':'Zone People Occupant Count','key':k,'label':f'Occ: {k}'} for k in occ_keys]
    # + [
    #     {'kind':'var','name':'Site Outdoor Air Drybulb Temperature','key':'Environment','label':'OAT'},
    #     {'kind':'var','name':'Site Outdoor Air Wetbulb Temperature','key':'Environment','label':'OWB'},
    # ]
)

# 3) plot covariance (pull any freq; we resample to 1H anyway)
fig = util.plot_sql_cov_heatmap(
    control_sels=control_sels,
    output_sels=output_sels,
    reporting_freq=None,     # <- don't filter out Zone Timestep rows
    resample='1h',           # compute cov on hourly series
    reduce='mean',
    stat='cov',              # or 'corr' if you want scale-free
    min_periods=12,
    include_design_days=False
)

#### Occupant plot

In [ ]:
# 1) discover zone keys that exist for the variable
occ_keys = (
    util.list_sql_zone_variables(
        name='Zone People Occupant Count',
        reporting_freq=None,            # don't filter; accept Zone Timestep, Hourly, etc.
        include_design_days=False
    )['KeyValue']
    .dropna().astype(str).tolist()
)

# (optional) limit to first N zones
# occ_keys = occ_keys[:8]

# 2) build selections and plot
selections = [
    {'kind':'var', 'name':'Zone People Occupant Count', 'key':k, 'label':k}
    for k in occ_keys
]

fig = util.plot_sql_series(
    selections=selections,
    reporting_freq=None,      # pull whatever is in the DB
    resample='1h',            # average to hourly; set to None for native timestep
    aggregate_vars='mean',    # hourly mean occupancy; use 'sum' for person-hours per hour
    title='Occupant Count per Zone',
    show=True
)

#### Other plots

In [ ]:
xp=EPlusSqlExplorer(sql_path="eplus_out/eplusout_kf_test.sqlite")

In [ ]:
# If you used the suggested test DB/table names:
df = xp.get_table_data(db="eplus_out/eplusout_kf_test.sqlite", table="KalmanEstimates")


In [ ]:
df

In [ ]:
zone1_df=df[df["Zone"]=="SPACE1-1"]
zone1_df

In [ ]:
import plotly.graph_objects as go

def plot_df_columns(df, x_column, trace_columns):
    """
    Generates a plotly line plot for specified columns in a DataFrame.

    Args:
        df: pandas DataFrame
        x_column: Name of the column to use for the x-axis.
        trace_columns: A list of column names to plot as separate traces on the y-axis.

    Returns:
        A plotly Figure object.
    """
    fig = go.Figure()

    for col in trace_columns:
        if col in df.columns:
            fig.add_trace(go.Scattergl(x=df[x_column], y=df[col], mode='lines', name=col))
        else:
            print(f"Warning: Column '{col}' not found in DataFrame.")

    fig.update_layout(
        title="Line Plot of DataFrame Columns",
        xaxis_title=x_column,
        yaxis_title="Value"
    )

    return fig

# Example usage with your 'zone1_df'
# Make sure 'Timestamp' is a datetime type for proper plotting
if 'Timestamp' in zone1_df.columns:
    zone1_df['Timestamp'] = pd.to_datetime(zone1_df['Timestamp'])

# Define the columns to plot
x_col = 'Timestamp'
y_cols = ['y_w', 'yhat_w', 'alpha_m', 'beta_w']

# Generate and show the plot
fig = plot_df_columns(zone1_df, x_col, y_cols)
fig.show()

In [ ]:
cols = ["Timestamp","Zone","y_T","yhat_T","alpha_T","beta_T"]
df = inspect_kf_sql(
    util,
    columns=cols,
    where="Zone='SPACE1-1' AND Timestamp >= '2013-12-01'",
    as_datetime=True
)

In [ ]:
df[:10].to_dict(orient='records')

In [ ]:
variable="System Node Mass Flow Rate"  #"System Node Temperature" #"Site Outdoor Air Barometric Pressure"#"Zone Inlet Air CO2 Concentration"#"Site Outdoor Air Relative Humidity" #"Zone Air Relative Humidity" #"Zone Outdoor Air Inlet Mass Flow Rate" #"Zone Infiltration Standard Density Volume Flow Rate" #'Site Outdoor Air Wetbulb Temperature' #"Zone Mean Air Temperature" #"Zone Air Relative Humidity" #"Zone Air CO2 Concentration" #"System Node Mass Flow Rate" #"Zone Air CO2 Concentration" #"Zone Air Relative Humidity" #"Zone Air Temperature"
# 4) discover keys and plot
display(util.list_sql_zone_variables(name=variable).head(10))

In [ ]:
zone_fig=util.plot_sql_zone_variable(
    variable,
    keys=["*"], #["SPACE1-1","SPACE2-1","SPACE3-1","SPACE4-1","SPACE5-1","PLENUM-1"], #["Environment"],#
    resample="1h",
    title=f"{variable} (Hourly Mean)"
)

In [ ]:
util.plot_sql_series([
    # {"kind":"var","name":"Zone Air CO2 Concentration","key":"SPACE1-1","label":"CO2 SPACE1-1"},
    {"kind":"var","name":"Air System Outdoor Air Mass Flow Rate","key":"*","label":"OA ṁ SPACE1-1"},
    # system-level node is also helpful (replace with your OA node key if different):
    {"kind":"var","name":"System Node Mass Flow Rate","key":"*","label":"OA node V̇"},
], reporting_freq=("TimeStep","Hourly"), resample="15min", meters_to_kwh=False)

In [ ]:
# What “Zone … Outdoor Air …” style vars exist?
util.list_sql_zone_variables(like="Zone %Outdoor Air%")

# Node-based flow variables (system-level). Then skim keys that look like OA nodes.
util.list_sql_zone_variables(name="System Node Mass Flow Rate")
util.list_sql_zone_variables(name="System Node Standard Density Volume Flow Rate")

# Controller/airloop scalar:
util.list_sql_zone_variables(name="Air System Outdoor Air Flow Fraction")


In [ ]:
# 1) Name of the schedule you actuate (created by prepare_run_with_co2)
sched = getattr(util, "_co2_outdoor_schedule", "CO2-Outdoor-Actuated")
print("Schedule:", sched)

# 2) Ensure SQL is produced and ask E+ to record that schedule’s value each timestep
util.ensure_output_sqlite()
util.ensure_output_variables([{
    "name": "Schedule Value",   # this is the reporting variable name
    "key":  sched,              # must match the schedule's name exactly
    "freq": "TimeStep",         # or "Hourly" if you prefer
}], activate=True, reset=True)

# 3) Run a quick design-day (or annual if you prefer)
util.run_design_day()  # or util.run_annual()

# 4) Plot from SQL
fig = util.plot_sql_series(
    selections=[{
        "kind": "var",
        "name": "Schedule Value",
        "key":  sched,
        "label": "Outdoor CO₂ [ppm]",
    }],
    reporting_freq=("TimeStep",),   # match what you requested
    include_design_days=False,
    resample="1H",                  # None for raw
    meters_to_kwh=False,
    title="Outdoor CO₂ Schedule (Actuated)",
    show=True,
)

In [ ]:
util.list_sql_zone_variables(name="Air System Outdoor Air Mass Flow Rate")

In [ ]:
# 3) See what meters actually have rows
for m in output_meteres:
    display(util.inspect_sql_meter(m, include_design_days=True))

# 4) Plot site electricity (facility)
elect_fig=util.plot_sql_meters(
    output_meteres,
    reporting_freq=("TimeStep","Hourly"),
    include_design_days=False,
    resample="1h",               # sum to hourly kWh
    meters_to_kwh=True,
    title=f"{', '.join(output_meteres)}"
)

# (optional) Net purchased if you enabled those two meters:
# elect_purchased=util.plot_sql_net_purchased_electricity(resample="1h")

In [ ]:
occ_keys = (
    util.list_sql_zone_variables(
        name='Zone People Occupant Count',
        reporting_freq=None,              # don't filter; show all
        include_design_days=False
    )['KeyValue']
    .dropna().astype(str).tolist()
)

In [ ]:
occ_keys = (
    util.list_sql_zone_variables(
        name='Zone People Occupant Count',
        reporting_freq=None,              # don't filter; show all
        include_design_days=False
    )['KeyValue']
    .dropna().astype(str).tolist()
)
occ_keys

In [ ]:
occ_keys

In [ ]:
fig = util.plot_sql_series(
    selections=output_sels,
    reporting_freq=None,      # pull whatever is in the DB
    resample='1h',            # average to hourly; set to None for native timestep
    aggregate_vars='mean',    # hourly mean occupancy; use 'sum' for person-hours per hour
    title='Occupant Count per Zone',
    show=True
)

In [ ]:
import simdkalman

In [ ]:
!pip install simdkalman

# SQLITE - Table Inspect

In [ ]:
xp=EPlusSqlExplorer(sql_path="eplus_out/eplusout_kf_test.sqlite")

In [ ]:
xp.list_tables()

# Documentation

https://energyplus.net/assets/nrel_custom/pdfs/pdfs_v25.1.0/EngineeringReference.pdf

https://energyplus.net/assets/nrel_custom/pdfs/pdfs_v24.1.0/InputOutputReference.pdf

# Sri Lanka Weather Data

https://climate.onebuilding.org/WMO_Region_2_Asia/LKA_Sri_Lanka/index.html

#HVAC Design

* https://www.mdpi.com/1996-1073/16/20/7124
* https://www.mdpi.com/2071-1050/17/5/1955
* https://discovery.ucl.ac.uk/id/eprint/10116413/1/manuscript%20baycal.pdf
* https://arxiv.org/pdf/2508.09118#:~:text=Page%202,(7)
* https://link.springer.com/article/10.1007/s12273-025-1300-4
* https://www.mdpi.com/2075-5309/13/2/314?
* (Bayes) https://www.sciencedirect.com/science/article/pii/S0306261925013017

A compact state-space model for a single zone with temperature, moisture, and $CO_2$ dynamics.


### **Zone Temperature (Sensible Energy)**
\begin{align}
C_s \dot T_z =
- UA\,T_z - c_{pa}(m_{inf}+m_{sa})T_z
\ +\ UA\,T_o + c_{pa} m_{inf} T_o + c_{pa} m_{sa} T_{sa}
+ Q_{bg} + f_c\,q^{occ}_{sens} N
\end{align}
where:
- $C_s$ = effective sensible thermal capacitance $[J/K]$
- $UA$ = overall heat transfer conductance $[W/K]$
- $c_{pa}$ = specific heat of air $[\approx 1006~J/(kg\,K)]$
- $m_{inf}$, $m_{sa}$ = outdoor- infiltration/supply air flow rates $[kg/s]$
- $T_o$, $T_z$, $T_{sa}$ = outdoor, zone, supply air temperatures $[^\circ C]$
- $Q_{bg}$, $q^{occ}_{sens}$ = background and per-person sensible heat gains $[W]$
- $f_c$ = convective fraction of sensible internal gain
- $N$ = number of occupants

***

### **Zone Humidity Ratio (Moisture)**
\begin{align}
M\dot\omega_z =
- (m_{inf} + m_{sa}) \omega_z
\ +\ m_{inf} \omega_o + m_{sa} \omega_{sa}
 + G_{bg} + g^{occ}_v N
\end{align}
where:
- $M$ = zone dry air mass or effective moisture capacity $$[kg_{dry}]$$
- $\omega_o,\, \omega_z,\, \omega_{sa}$ = outdoor, zone, supply air humidity ratios $[kg/kg_{dry}]$
- $G_{bg}$, $g^{occ}_v$ = background and per-person vapor gains $$[kg/s]$$

***

### **Zone $CO_2$ Concentration**
\begin{align}
M\dot c_z =
- (m_{inf} + m_{sa}) c_z
\ +\ m_{inf} c_o + m_{sa} c_{sa}
 + g_{CO2}^{occ} N
\end{align}
where:
- $c_o,\, c_z,\, c_{sa}$ = outdoor, zone, supply air $CO_2$ concentrations $[kg/kg_{dry}]$
- $g^{occ}_{CO2}$ = per-person $CO_2$ generation rate $[kg/s]$.

***

These forms are **physically complete, transparent, and directly express the conservation of energy, mass (moisture), and tracer (CO₂) for an air-conditioned zone** with standard HVAC inputs and disturbances.[6][7]

**References:**  
-  "Physical Model of an Air-Conditioned Space for Control Analysis," AIVC, see equations for energy and mass balances.[6]
-  "Cooling Load Calculations and Principles," CED Engineering, see zone and air mass conservation sections.[7]

[1](https://hvacvn.com/wp-content/uploads/2015/10/hvac-equations-data-and-rules-of-thumb.pdf)
[2](https://hvacsimplified.in/wp-content/uploads/2022/02/All-In-One-HVAC-Calculation-Formulas.pdf)
[3](https://www.pdhonline.com/courses/m378/m378content.pdf)
[4](https://www.av8rdas.com/uploads/1/0/3/2/103277290/supplemental_information_appendix_01_-_useful_equations_r6.pdf)
[5](https://www.scribd.com/document/505484310/HVAC-Simplified)
[6](https://www.aivc.org/sites/default/files/airbase_12998.pdf)
[7](https://www.cedengineering.com/userfiles/M06-004%20-%20Cooling%20Load%20Calculations%20and%20Principles%20-%20US.pdf)
[8](https://apps.dtic.mil/sti/tr/pdf/ADA196408.pdf)
[9](https://orbi.uliege.be/bitstream/2268/771/1/BSjournal_BS08B001_final_SB081028.pdf)

---

### Steady state equations

\begin{align}
T_z^\star
&=\frac{UA\,T_o\;+\;c_{pa}\,m_{\inf}\,T_o\;+\;c_{pa}\,m_{sa}\,T_{sa}\;+\;Q_{bg}\;+\;f_c\,q^{occ}_{sens}N}
{UA\;+\;c_{pa}\,(m_{sa}+m_{\inf})}\\
\omega_z^\star
&=\frac{m_{\inf}\,\omega_o\;+\;m_{sa}\,\omega_{sa}\;+\;G_{bg}\;+\;g^{occ}_{v}N}
{m_{sa}+m_{\inf}}\\
c_z^\star
&=\frac{m_{\inf}\,c_o\;+\;m_{sa}\,c_{sa}\;+\;g^{occ}_{CO2}N}
{m_{sa}+m_{\inf}}
\end{align}


## Uncertainty Model

Re-arranging the steady state equations we have
\begin{align}
T_z^\star
&=\frac{UA+{c_{pa}}m_{\inf}}{UA+{c_{pa}}(m_{sa}+m_{\inf})}\,(T_o-T_{sa})\;+\;T_{sa}\;+\;\frac{Q_{bg}\;+\;f_c\,q^{occ}_{sens}N}{UA\;+\;c_{pa}\,(m_{sa}+m_{\inf})}\\
\omega_z^\star
&=\frac{m_{inf}}{m_{sa}+m_{inf}}\,(\omega_o-\omega_{sa})\;+\;\omega_{sa}\;+\;\frac{G_{bg}\;+\;g^{occ}_{v}N}{m_{sa}+m_{\inf}}\\
c_z^\star
&=\frac{m_{inf}}{m_{sa}+m_{inf}}\,(c_o-c_{sa})\;+\;c_{sa}\;+\;\frac{g^{occ}_{CO2}N}
{m_{sa}+m_{\inf}}
\end{align}

---

Let
\begin{align}
y\triangleq &=\begin{bmatrix}
y_T \\   y_{\omega}\\ y_{CO2}
\end{bmatrix}
= \begin{bmatrix}
T_z^\star\\
\omega_z^\star\\
c_z^\star
\end{bmatrix}
\end{align}

---

\begin{align}
\phi &= \begin{bmatrix}
(T_o - T_{sa}) & 1 & 0 & 0 & 0 \\
0 & 0 & (\omega_o - \omega_{sa}) & 1 &0 \\
0 & 0 & (c_o - c_{sa}) & 0 & 1\\
\end{bmatrix}\\
\beta &= \begin{bmatrix}
\frac{UA+{c_{pa}}m_{\inf}}{UA+{c_{pa}}(m_{sa}+m_{\inf})} \\
T_{sa}+\frac{Q_{bg}\;+\;f_c\,q^{occ}_{sens}N}{UA\;+\;c_{pa}\,(m_{sa}+m_{\inf})}\\
\frac{m_{inf}}{m_{sa}+m_{inf}}\\
\omega_{sa}\;+\;\frac{G_{bg}\;+\;g^{occ}_{v}N}{m_{sa}+m_{\inf}}\\
c_{sa}\;+\;\frac{g^{occ}_{CO2}N}
{m_{sa}+m_{\inf}}
\end{bmatrix}
\end{align}

Note that the following inversion holds

\begin{align}
m_{inf} &= \frac{\beta_{3}\,m_{sa}}{1 - \beta_{3}}\\
UA  &= c_{pa}\ m_{sa}\left(\frac{\beta_{1}}{1 - \beta_{1}}-\frac{\beta_{3}}{1 - \beta_{3}}\right)\\
Q_{bg}\;+\;f_c\,q^{occ}_{sens}N&=(\beta_{2}-T_{sa})\left(\frac{c_{pa}\ m_{sa}}{1 - \beta_{1}}\right)\\
G_{bg} + g^{occ}_v N & = (\beta_4 - \omega_{sa})\,\frac{m_{sa}}{1 - \beta_{3}}\\
g^{occ}_{CO2} N &= (\beta_5 - c_{sa})\,\frac{m_{sa}}{1 - \beta_{3}}
\end{align}

---

\begin{align}
y = \phi \beta + \epsilon, \qquad \epsilon \sim \mathscr{N}(0,\Sigma_R)
\end{align}

##Modelling and Control

---

Let
\begin{align}
y_k\triangleq &\begin{bmatrix}
y_{T,k} \\   y_{\omega,k}\\ y_{CO2,k}
\end{bmatrix}
= \begin{bmatrix}
T_{z,k}^\star\\
\omega_{z,k}^\star\\
c_{z,k}^\star
\end{bmatrix}
\end{align}

---

\begin{align}
\phi_k &= \begin{bmatrix}
(T_{o,k} - T_{sa,k}) & 1 & 0 & 0 & 0 \\
0 & 0 & (\omega_{o,k} - \omega_{sa,k}) & 1 &0 \\
0 & 0 & (c_{o,k} - c_{sa,k}) & 0 & 1\\
\end{bmatrix}
\end{align}


\begin{align}
\beta_{k} & = \beta_{k-1}+ w_k ,\qquad
w_k \sim \mathscr{N}(0,\Sigma_P),\\
y_k &= \phi_k \beta_k  +\varepsilon_k,\qquad
\varepsilon_k \sim \mathscr{N}(0,\Sigma_R),
\end{align}
where $(\Sigma_R=\operatorname{diag}(\sigma_T^2,\ \sigma_\omega^2,\ \sigma_c^2))$ are the (unknown) error variances for the three equations.

---

The Kalman filter
\begin{align}
&\text{Predict: } \mu^-=\mu_{k-1},\ \Sigma^-=\Sigma_{k-1}+\Sigma_P\\
&\text{Gain: } K_k=\Sigma^- \phi_k^\top(\phi_k\Sigma^- \phi_k^\top+\Sigma_R)^{-1}\\
&\text{Update: } \mu_k=\mu^- + K_k(y_k-\phi_k\mu^-),\ \Sigma_k=(I-K_k\phi_k)\Sigma^-.
\end{align}

---

\begin{align}
\widehat{y}_k &\triangleq \phi_k\mu_k
\end{align}

Let
\begin{align}
\mu_k =\begin{bmatrix}\widehat{T}^\star_{z,k}\\ \widehat{\omega}^\star_{z,k}\\\widehat{c}^\star_{z,k}
\end{bmatrix}
\end{align}

Here are generalized expressions ** for an HVAC system with **air mixing, cooling/heating coil, and reheat**:

***

### **Air Mixing (Pre-coil)**
Assume the air handler mixes **outdoor air (OA)** and **return air (RA)**:
- $ m_{oa,k} $: outdoor air mass flow
- $ m_{ra,k} $: return air mass flow
- $ m_{ma,k} = m_{oa,k} +m_{ra,k}$: total mixed air mass flow

The **mixed air properties** entering the coil are:

\begin{align}
T_{ma,k} &= \frac{m_{oa,k} T_{o,k} + m_{ra,k} T^\star_{z,k}}{m_{ma,k}},\\
\omega_{ma,k} &= \frac{m_{oa,k}\omega_{o,k} + m_{ra,k}\omega^\star_{z,k}}{m_{ma,k}},\\
c_{m,k} &= \frac{m_{oa,k}c_{o,k} + m_{ra,k}c^\star_{z,k}}{m_{ma,k}}
\end{align}

Let $\gamma_{m,k}=\frac{m_{o,k}}{m_{oa,k}+m_{ra,k}}$ then
\begin{align}
T_{ma,k} &= m_{ma,k}\left(\gamma_{m,k} T_{o,k} + (1-\gamma_{m,k}) T^\star_{z,k}\right),\\
\omega_{ma,k} &= m_{ma,k}\left(\gamma_{m,k}\omega_{o,k} + (1-\gamma_{m,k})\omega^\star_{z,k}\right),\\
c_{ma,k} &= m_{ma,k}\left(\gamma_{m,k}c_{o,k} + (1-\gamma_{m,k})c^\star_{z,k}\right)
\end{align}

---



### Sensible (Thermal) Energy Required by the Coil

For each time step, the energy rate required to move the mixed air from **mixed condition ($T_{ma,k}$)** to the **coil outlet or supply air temperature (${T}_{sa,k+1}$)** is:
\begin{align}
Q_{\text{coil},k+1} = {m}_{ma,k} \, c_{pa} |({T}_{\text{sa},k+1} - T_{\text{ma},k})|
\end{align}
- ${T}_{\text{sa},k+1}$: supply air temperature [°C] for the next step
- $T_{\text{ma},k}$: mixed air temperature entering coil [°C]

***

### Latent (Moisture) Energy Cost

If moisture is **removed** (dehumidification by cooling coil):
\begin{align}
Q_{\text{latent},k} &= {m}_{ma,k} \cdot h_{fg} \cdot \max(0, \omega_{\text{ma},k} - {\omega}^\star_{z,k+1})
\end{align}

- $h_{fg}$: latent heat of vaporization (≈ 2,500,000 J/kg at room temperature)
- $\omega_{\text{ma},k}$: mixed air humidity ratio (kg water/kg dry air)
- ${\omega}_{\text{sa},k+1}$: supply air humidity ratio at the next step

(Coils don't **add** moisture, so this term is only positive when $\omega_{sa,k} < \omega_{ma,k+1}$.)

***

### Combined Coil Power (Cooling or Heating)

The **total coil load at time $k$** (assuming no reheat for simplicity) is:
\begin{align}
Q_{\text{coil,total},k+1} &= Q_{\text{coil},k+1} + Q_{\text{latent},k+1},\\
&= {m}_{ma,k} \left[c_{pa}|({T}_{\text{sa},k+1} - T_{\text{ma},k})| + h_{fg} \cdot \max(0, \omega_{\text{ma},k} - {\omega}_{\text{sa},k+1}) \right]\\
&= {m}_{ma,k} \left[c_{pa}|\left({T}_{\text{sa},k+1} - \left(\gamma_{m,k} T_{o,k} + (1-\gamma_{m,k}) T^\star_{z,k}\right)\right)| + h_{fg} \cdot \max\left(0, \left(\gamma_{m,k}\omega_{o,k} + (1-\gamma_{m,k})\omega^\star_{z,k}\right) - {\omega}_{\text{sa},k+1}\right) \right]
\end{align}

Substituting the Kalman filter estimate $\mu_k$
\begin{align}
\widehat{Q}_{\text{coil,total},k+1} &= {m}_{ma,k} \left[c_{pa}|\left({T}_{\text{sa},k+1} - \left(\gamma_{m,k} T_{o,k} + (1-\gamma_{m,k}) \widehat{T}^\star_{z,k}\right)\right)| + h_{fg} \cdot \max\left(0, \left(\gamma_{m,k}\omega_{o,k} + (1-\gamma_{m,k})\widehat{\omega}^\star_{z,k}\right) - {\omega}_{\text{sa},k+1}\right) \right]
\end{align}

The optimal control HVAC problem is to then find $(T_{sa,k+1},{\omega}_{{sa},k+1})\in D$ and $\gamma_{m,k}\in [0,1]$ such that $\widehat{Q}_{\text{coil,total},k+1}$ is minimized.

***

### Supply mass flowrate

Predicted zone temperature and moisture at the next time step.
\begin{align}
T^\star_{z,k+1}\approx \beta_{1,k}(T_{o,k+1}-T_{sa,k+1})+\beta_{2,k},\\
\omega^\star_{z,k+1}\approx \beta_{3,k}(\omega_{o,k+1}-\omega_{sa,k+1})+\beta_{4,k}
\end{align}

* Check if $(T^\star_{z,k+1},\omega^\star_{z,k+1})\in D$
* If not increase $m_{sa,k+1}$

---

### Distributed Control of Multi-zone HVAC Systems Considering Indoor Air Quality

https://arxiv.org/pdf/2003.08208